## Twin Analysis

The HCP databases uses a twin sample, allowing for the comparison of monozygotic (MZ) vs dizygotic (DZ) twins. 

This allows us to compare the heritability of oscillations.

In [1]:
# Import required libraries/functions
from __future__ import print_function, division
import random
import itertools
import numpy as  np

# Import custom code from module om, including general functions and OO code for handling data
from om.core.db import OMDB
from om.core.osc import Osc
from om.core.io import load_meg_pairs
from om.core.utils import check_file_status
from om.meg.twin import *

from om.meg.twin import _comp_pf_pair, _comp_space_pair, _comp_osc_pair, _comp_sl_pair

# TODO: Make all this duplicated runs for 3 subj groups into functions.

In [2]:
# Pull out twin data from demographics file
mz_twins, dz_twins, all_twins, not_twins = get_twin_data()

# Match twin pairs
mz_id_pairs, mz_singles = match_twins(mz_twins)
dz_id_pairs, dz_singles = match_twins(dz_twins)

# Check how many pairs where extracted
print('There are', str(len(mz_id_pairs)), 'MZ twin pairs.')
print('There are', str(len(dz_id_pairs)), 'DZ twin pairs.')

There are 18 MZ twin pairs.
There are 19 DZ twin pairs.


In [3]:
# Get database object and set up database to use
db = OMDB()
dat_source = 'HCP'

# Check which twin subjects are available
av_dat, no_dat = check_file_status(all_twins, db, dat_source='HCP')

Of 93 subjects, 77 files are available and 16 files are unavailable.


In [4]:
# Given the missing data, check which twin pairs are complete
mz_complete_pairs = check_complete_pairs(mz_id_pairs, av_dat)
dz_complete_pairs = check_complete_pairs(dz_id_pairs, av_dat)

# Print out number of available pairs by twin type
print('There are', str(len(mz_complete_pairs)), 'complete MZ twin pairs.')
print('There are', str(len(dz_complete_pairs)), 'complete DZ twin pairs.')

There are 10 complete MZ twin pairs.
There are 16 complete DZ twin pairs.


In [5]:
# Check which pairs are missing (at least one) subject
print('MZ Twins Incomplete Pairs:')
print(list(set(mz_id_pairs) - set(mz_complete_pairs)))
print('DZ Twins Incomplete Pairs:')
print(list(set(dz_id_pairs) - set(dz_complete_pairs)))

MZ Twins Incomplete Pairs:
[(189349, 500222), (151526, 352132), (149741, 433839), (287248, 660951), (187547, 200109), (104012, 153732), (102816, 680957), (191033, 872764)]
DZ Twins Incomplete Pairs:
[(192641, 825048), (182840, 205119), (108323, 140117)]


In [6]:
# Get a list of all possible non-twin pairs, to compare to
all_pairs = list(itertools.combinations(av_dat, 2))
non_twin_pairs = rm_twin_pairs(all_pairs, mz_id_pairs + dz_id_pairs)

# Of all possible combinations, get a random sample of non-twin pairs, matching number of twin pairs
n_non_twin = len(dz_complete_pairs)
rand_inds = random.sample(range(len(non_twin_pairs)), n_non_twin)
non_twin_pairs = [non_twin_pairs[i] for i in rand_inds]

## Load Data

In [7]:
# Get database object and oscillatory band definition
db = OMDB()
osc = Osc(default=True)

In [8]:
# Load subjects into pairs
mz_twin_dat = load_meg_pairs(mz_complete_pairs[0:2], osc_bands_vert=True, osc=osc, db=db)
dz_twin_dat = load_meg_pairs(dz_complete_pairs[0:2], osc_bands_vert=True, osc=osc, db=db)
non_twin_dat = load_meg_pairs(non_twin_pairs[0:2], osc_bands_vert=True, osc=osc, db=db)

## Peak Frequencies

Compares the peak frequencies within oscillatory bands between twins and non-twins.

In [30]:
# Settings
peak_type = 'band'
avg = 'median'

# Compare peak frequencies
mz_peak_avg, mz_peak_dat = comp_peak_freq(mz_twin_dat, peak_type=peak_type, avg=avg)
dz_peak_avg, dz_peak_dat = comp_peak_freq(dz_twin_dat, peak_type=peak_type, avg=avg)
non_twin_peak_avg, non_twin_peak_dat = comp_peak_freq(non_twin_dat, peak_type=peak_type, avg=avg)

In [31]:
# Compare MZ twins - Difference in Oscillatory Band Peak Frequencies

# Initialize output data variable
mz_peak_dat_OLD = np.empty(shape=[0, osc.n_bands])

# Loop through MZ twin pairs, comparing them
for pair in mz_complete_pairs[0:2]:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    #mz_peak_dat_OLD = np.vstack([mz_peak_dat_OLD, compare_peak_freqs_pair(dat, avg='median')])
    mz_peak_dat_OLD = np.vstack([mz_peak_dat_OLD, _comp_pf_pair(dat, avg='median')])
    
# Get average difference of peak frequencies across pairs
mz_peak_avg_OLD = np.mean(mz_peak_dat, axis=0, keepdims=True)

In [32]:
# Compare DZ twins - Difference in Oscillatory Band Peak Frequencies

# Initialize output data variable
dz_peak_dat_OLD = np.empty(shape=[0, osc.n_bands])

# Loop through MZ twin pairs, comparing them
for pair in dz_complete_pairs[0:2]:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    #dz_peak_dat_OLD = np.vstack([dz_peak_dat_OLD, compare_peak_freqs_pair(dat, avg='median')])
    dz_peak_dat_OLD = np.vstack([dz_peak_dat_OLD, _comp_pf_pair(dat, avg='median')])
    
# Get average difference of peak frequencies across pairs
dz_peak_avg_OLD = np.mean(dz_peak_dat, axis=0, keepdims=True)

In [33]:
# Compare non-twins - Difference in Oscillatory Band Peak Frequencies

# Initialize output data variable
non_twin_peak_dat_OLD = np.empty(shape=[0, osc.n_bands])

# Loop through non-twin pairs, comparing them
for pair in non_twin_pairs[0:2]:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    #non_twin_peak_dat_OLD = np.vstack([non_twin_peak_dat_OLD, compare_peak_freqs_pair(dat, avg='median')]) 
    non_twin_peak_dat_OLD = np.vstack([non_twin_peak_dat_OLD, _comp_pf_pair(dat, avg='median')]) 

# Get average difference of peak frequencies across pairs
non_twin_peak_avg_OLD = np.mean(non_twin_peak_dat_OLD, axis=0, keepdims=True)

In [34]:
# Print out Results
print('MZ Twin Oscillatory Band Peak Frequency Results: ')
print_twin_results_NEW(mz_peak_avg, osc.labels)
print('DZ Twin Oscillatory Band Peak Frequency Results: ')
print_twin_results_NEW(dz_peak_avg, osc.labels)
print('Non-Twin Oscillatory Band Peak Frequency Results: ')
print_twin_results_NEW(non_twin_peak_avg, osc.labels)

MZ Twin Oscillatory Band Peak Frequency Results: 
	 Theta 	 :  0.4258
	 Alpha 	 :  0.3895
	 Beta 	 :  1.9886
	 LowGamma 	 :  1.1161
DZ Twin Oscillatory Band Peak Frequency Results: 
	 Theta 	 :  1.2450
	 Alpha 	 :  1.2445
	 Beta 	 :  1.6689
	 LowGamma 	 :  1.3045
Non-Twin Oscillatory Band Peak Frequency Results: 
	 Theta 	 :  0.7168
	 Alpha 	 :  0.6918
	 Beta 	 :  3.4031
	 LowGamma 	 :  0.7627


### Oscillatory Band - Spatial Overlap

Compares the spatial topography of oscillatory bands between pairs of subject. 

In [10]:
# Compare spatial overlap
mz_space_avg, mz_space_dat = comp_osc_space(mz_twin_dat)
dz_space_avg, dz_space_dat = comp_osc_space(dz_twin_dat)
non_twin_space_avg, non_twin_space_dat = comp_osc_space(non_twin_dat)

In [9]:
# Compare MZ twins - Oscillatory Band Spatial Overlap

# Initialize output data variable
mz_space_dat_OLD = np.empty(shape=[0, osc.n_bands])

# Loop through MZ twin pairs, comparing them
for pair in mz_complete_pairs[0:2]:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    mz_space_dat_OLD = np.vstack([mz_space_dat_OLD, compare_spatial_pair(dat)])
    
# Get average correlation within bands across pairs
mz_avg_space_OLD = np.mean(mz_space_dat_OLD, axis=0, keepdims=True)

In [10]:
# Compare MZ twins - Oscillatory Band Spatial Overlap

# Initialize output data variable
dz_space_dat_OLD = np.empty(shape=[0, osc.n_bands])

# Loop through MZ twin pairs, comparing them
for pair in dz_complete_pairs[0:2]:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    dz_space_dat_OLD = np.vstack([dz_space_dat_OLD, compare_spatial_pair(dat)])
    
# Get average correlation within bands across pairs
dz_avg_space_OLD = np.mean(dz_space_dat_OLD, axis=0, keepdims=True)

In [11]:
# Compare non-twins - Oscillatory Band Center Frequencies

# Initialize output data variable
non_twin_space_dat_OLD = np.empty(shape=[0, osc.n_bands])

# Loop through non-twin pairs, comparing them
for pair in non_twin_pairs[0:2]:
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    non_twin_space_dat_OLD = np.vstack([non_twin_space_dat_OLD, compare_spatial_pair(dat)]) 
    
# Get average correlation within bands across pairs
non_twin_avg_space_OLD = np.mean(non_twin_space_dat_OLD, axis=0, keepdims=True)

In [12]:
# Print out Results
print('MZ Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results(mz_avg_space.T, osc.labels)
print('DZ Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results(dz_avg_space.T, osc.labels)
print('Non-Twin Oscillatory Band Spatial Overlap Results: ')
print_twin_results(non_twin_avg_space.T, osc.labels)

MZ Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7723
	 Alpha 	 :  0.6776
	 Beta 	 :  0.9506
	 LowGamma 	 :  0.6287
DZ Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7450
	 Alpha 	 :  0.7027
	 Beta 	 :  0.8878
	 LowGamma 	 :  0.6114
Non-Twin Oscillatory Band Spatial Overlap Results: 
	 Theta 	 :  0.7436
	 Alpha 	 :  0.4967
	 Beta 	 :  0.9231
	 LowGamma 	 :  0.5849


### Oscillation Parameters Comparison

The following compares oscillatory parameters, such as center frequency, power and bandwidth from within oscillatory bands. 

NOTE: As implemented, the these comparisons are not specific to how subjects are similar/different. 
Subjects can differ in either:
- The spatial topography across which the oscillation band is found and/or
- The oscillatory parameter within the vertices where the oscillatory band is found

^ This can/will be parcelled out in further analyses:
- A basic check is simply to look at the degree of overlap of oscillatory band topographies between subjects. 

In [15]:
# Settings. Osc-Param: {0: CF, 1: Power, 2: BW}
osc_param = 0

# Compare oscillatory parameters
mz_param_avg, mz_param_dat = comp_osc_param(mz_twin_dat, osc_param)
dz_param_avg, dz_param_dat = comp_osc_param(dz_twin_dat, osc_param)
non_twin_param_avg, non_twin_param_dat = comp_osc_param(non_twin_dat, osc_param)

In [14]:
# Compare MZ twins - Oscillatory Band Center Frequencies

# Initialize variable to store output data
mz_param_dat_OLD = np.zeros([len(mz_complete_pairs), osc.n_bands, 2])

# Loop through MZ twin pairs, comparing them
for ind, pair in enumerate(mz_complete_pairs[0:2]):
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    #mz_corr_dat[ind, :, :] = compare_osc_param_pair(dat, osc_param)
    mz_param_dat_OLD[ind, :, :] = _comp_osc_pair(dat, osc_param)
    
# Get average correlation within bands across pairs
mz_param_avg_OLD = np.mean(mz_param_dat_OLD, axis=0)

In [15]:
# Compare DZ twins - Oscillatory Band Center Frequencies

# Initialize variable to store output data
dz_param_dat_OLD = np.zeros([len(dz_complete_pairs), osc.n_bands, 2])

# Loop through DZ twin pairs, comparing them
for ind, pair in enumerate(dz_complete_pairs[0:2]):
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    #dz_corr_dat[ind, :, :] = compare_osc_param_pair(dat, osc_param)
    dz_param_dat_OLD[ind, :, :] = _comp_osc_pair(dat, osc_param)
    
# Get average correlation within bands across pairs
dz_param_avg_OLD = np.mean(dz_param_dat_OLDt, axis=0)

In [16]:
# Compare non-twins - Oscillatory Band Center Frequencies

# Initialize variable to store output data
non_twin_param_dat_OLD = np.zeros([len(non_twin_pairs), osc.n_bands, 2])

# Loop through non-twin pairs, comparing them
for ind, pair in enumerate(non_twin_pairs[0:2]):
    dat = load_meg_list(pair, osc_bands_vert=True, osc=osc, db=db, dat_source='HCP')
    #non_twin_corr_dat[ind, :, :] = compare_osc_param_pair(dat, osc_param)
    non_twin_param_dat_OLD[ind, :, :] = _comp_osc_pair(dat, osc_param)
    
# Get average correlation within bands across pairs
non_twin_param_avg_OLD = np.mean(non_twin_param_dat_OLD, axis=0)

In [16]:
# Print out results
print('MZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(mz_avg_corr, osc.labels)
print('DZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(dz_avg_corr, osc.labels)
print('Non-Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(non_twin_avg_corr, osc.labels)

MZ Twin Oscillatory Band Center Frequency Results: 


NameError: name 'mz_avg_corr' is not defined

In [27]:
# Print out results
print('MZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(mz_param_avg, osc.labels)
print('DZ Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(dz_param_avg, osc.labels)
print('Non-Twin Oscillatory Band Center Frequency Results: ')
print_twin_results(non_twin_param_avg, osc.labels)

MZ Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.0780
	 Alpha 	 :  0.1183
	 Beta 	 :  0.1200
	 LowGamma 	 :  0.1304
DZ Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.0782
	 Alpha 	 :  -0.0372
	 Beta 	 :  0.0477
	 LowGamma 	 :  0.0072
Non-Twin Oscillatory Band Center Frequency Results: 
	 Theta 	 :  0.0170
	 Alpha 	 :  0.0679
	 Beta 	 :  0.1222
	 LowGamma 	 :  0.1507


In [29]:
np.mean(mz_param_dat, axis=0, keepdims=False)

array([[  7.80496781e-02,   1.37223756e-04],
       [  1.18286098e-01,   4.38291445e-09],
       [  1.20020628e-01,   5.92841812e-04],
       [  1.30384440e-01,   1.54533287e-02]])

## Slope Comparison

In [12]:
# Compare slopes
mz_slope_avg, mz_slope_dat = comp_slope(mz_twin_dat)
dz_slope_avg, dz_slope_dat = comp_slope(dz_twin_dat)
non_twin_slope_avg, non_twin_slope_dat = comp_slope(non_twin_dat)

In [18]:
# Check how many twin pairs are available
n_mz_pairs = len(mz_complete_pairs)
mz_slope_dat_OLD = np.zeros([n_mz_pairs, 2])

# Loop through MZ twin pairs, comparing them
for ind, pair in enumerate(mz_complete_pairs[0:2]):
    dat = load_meg_list(pair, db=db, dat_source='HCP')
    #mz_sl_corr_dat[ind, :] = compare_slope_pair(dat)
    mz_slope_dat_OLD[ind, :] = _comp_sl_pair(dat)

# Get average correlation within bands across pairs
mz_slope_avg_OLD = np.mean(mz_slope_dat_OLD, axis=0, keepdims=True)

In [19]:
# Check how many twin pairs are available
n_dz_pairs = len(dz_complete_pairs)
dz_sl_corr_dat = np.zeros([n_dz_pairs, 2])

# Loop through MZ twin pairs, comparing them
for ind, pair in enumerate(dz_complete_pairs[0:2]):
    dat = load_meg_list(pair, db=db, dat_source='HCP')
    #dz_sl_corr_dat[ind, :] = compare_slope_pair(dat)
    dz_slope_dat_OLD[ind, :] = _comp_sl_pair(dat)

# Get average correlation within bands across pairs
dz_slope_avg_OLD = np.mean(dz_slope_dat_OLD, axis=0, keepdims=True)

In [20]:
# Compare non-twins - Oscillatory Band Center Frequencies

# Get a random sample of non-twin pairs, matching number of twin pairs
#n_non_twin = n_dz_pairs
#rand_inds = random.sample(range(len(non_twin_pairs)), n_non_twin)
#non_twin_samp = [non_twin_pairs[i] for i in rand_inds]
non_twin_slope_dat_OLD = np.zeros([n_non_twin, 2])

# Loop through non-twin pairs, comparing them
for ind, pair in enumerate(non_twin_samp):
    dat = load_meg_list(pair, db=db, dat_source='HCP')
    #non_twin_sl_corr_dat[ind, :] = compare_slope_pair(dat)
    non_twin_slope_dat_OLD[ind, :] = _comp_sl_pair(dat)
    
# Get average correlation within bands across pairs
non_twin_slope_avg_OLD = np.mean(non_twin_slope_dat_OLD, axis=0, keepdims=True)

In [20]:
# Print out results
print('MZ Twin Slope Results: ')
print_twin_results_NEW(mz_slope_avg, ['Slope'])
print('DZ Twin Slope Results: ')
print_twin_results_NEW(dz_slope_avg, ['Slope'])
print('Non-Twin Slope Results: ')
print_twin_results_NEW(non_twin_slope_avg, ['Slope'])

MZ Twin Slope Results: 
	 Slope 	 :  0.2097
DZ Twin Slope Results: 
	 Slope 	 :  0.2535
Non-Twin Slope Results: 
	 Slope 	 :  0.3165


In [ ]:
# Print out results
print('MZ Twin Slope Results: ')
print_twin_results(mz_avg_sl_corr, ['Slope'])
print('DZ Twin Slope Results: ')
print_twin_results(dz_avg_sl_corr, ['Slope'])
print('Non-Twin Slope Results: ')
print_twin_results(non_twin_avg_sl_corr, ['Slope'])

In [25]:
np.mean(mz_slope_dat, axis=0, keepdims=False)

array([  2.09715381e-01,   2.96113410e-12])

## TEST CODE

In [31]:
ind = 1
osc = Osc(default=True)

meg_subj_1 = MegData(db, 'HCP', osc)
meg_subj_2 = MegData(db, 'HCP', osc)

meg_subj_1.import_foof(mz_complete_pairs[ind][0])
meg_subj_1.osc_bands_vertex()
meg_subj_1.peak_freq('band')
meg_subj_2.import_foof(mz_complete_pairs[ind][1])
meg_subj_2.osc_bands_vertex()
meg_subj_2.peak_freq('band')

In [32]:
print(meg_subj_1.peaks)
print(meg_subj_2.peaks)

{'Theta': 5.8272859900125198, 'Beta': 18.316600967202131, 'LowGamma': 36.19577783790033, 'Alpha': 11.286065038458034}
{'Theta': 5.8353296892487636, 'Beta': 21.409984315998873, 'LowGamma': 34.504582957991857, 'Alpha': 10.72397369749755}


In [ ]:
n_rand_pairs = len(rand_pairs)
rand_corr_dat = np.zeros([n_rand_pairs, 4, 2])


for ind, pair in enumerate(rand_pairs):
    rand_corr_dat[ind, :, :] = compare_twin_pair(pair)

In [ ]:
rand_pairs = [(181232, 166438), (212318, 175237), (204521, 255639),
              (191841, 293748), (214524, 352738), (223929, 111514)]

In [7]:
pair_ind = 4

# Set subject number to load
subj_1 = mz_complete_pairs[pair_ind][0]
subj_2 = mz_complete_pairs[pair_ind][1]

meg_

In [ ]:
np.mean(mz_corr_dat, axis=0)

In [ ]:
np.mean(dz_corr_dat, axis=0)

In [ ]:
np.mean(rand_corr_dat, axis=0)

In [ ]:
np.mean(non_twin_corr_dat, axis=0)